<a href="https://colab.research.google.com/github/Noman-ashraf1/Paraphrase-Generation-using-T5-Pre-trained-model/blob/main/Paraphrase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#import libraries

In [2]:
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, trainer_callback
import os
import re


#Load Dataset

In [3]:
df = load_dataset('paws', 'labeled_final')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.79k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [4]:
df['train']

Dataset({
    features: ['id', 'sentence1', 'sentence2', 'label'],
    num_rows: 49401
})

In [5]:
data = pd.DataFrame(df["train"])
data.head()

,id,sentence1,sentence2,label
0,1,"In Paris , in October 1560 , he secretly met t...","In October 1560 , he secretly met with the Eng...",0
1,2,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
2,3,"There are also specific discussions , public p...","There are also public discussions , profile sp...",0
3,4,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,5,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1


In [6]:
data = data[data['label']==1]
data.head()

,id,sentence1,sentence2,label
1,2,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
3,4,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,5,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1
5,6,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro , ...",1
7,8,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...,1


In [7]:
import pandas as pd

# Extract data directly from the Hugging Face Dataset to pandas DataFrame
train_data = df['train'].shuffle(seed=42).select(range(3000))
test_data = df['test'].shuffle(seed=42).select(range(300))
valid_data = df['validation'].shuffle(seed=42).select(range(300))

# Convert to pandas DataFrames and add 'input_data' and 'target' columns
train_df = pd.DataFrame({
    'input_data': ['paraphrase: ' + item['sentence1'] for item in train_data],
    'target': [item['sentence2'] for item in train_data]
})

test_df = pd.DataFrame({
    'input_data': ['paraphrase: ' + item['sentence1'] for item in test_data],
    'target': [item['sentence2'] for item in test_data]
})

valid_df = pd.DataFrame({
    'input_data': ['paraphrase: ' + item['sentence1'] for item in valid_data],
    'target': [item['sentence2'] for item in valid_data]
})




In [8]:
train_df

,input_data,target
0,paraphrase: Allied armies invaded France in ea...,"In early 1814 , Allied armies entered France ,..."
1,paraphrase: The Geamărtălui River is a tributa...,The river Geamărtălui is a tributary of the St...
2,paraphrase: The original 159 Squadron was to b...,The original squadron 159 was to be dissolved ...
3,paraphrase: Mirambeau is located on Via Turone...,"Mirambeau is situated on Via Turonensis , the ..."
4,paraphrase: Ryszard Kaczorowski received symbo...,( Lech Wałęsa accepted the symbols of Ryszard ...
...,...,...
2995,paraphrase: PremLata Singh was married on June...,Birender Singh married the PremLata Singh on J...
2996,paraphrase: She is born on 18 April 1976 in Us...,"She was born in Usera , Spain ( Madrid ) on Ap..."
2997,paraphrase: Its capital was Nuuk ( modern Godt...,Its capital was at Nuuk ( modern Godthaab ) .
2998,paraphrase: The Rusca River is a tributary of ...,The river Suceviţa is a tributary of the Rusca...


In [9]:
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)
valid_data = Dataset.from_pandas(valid_df)
train_data[0]

{'input_data': 'paraphrase: Allied armies invaded France in early 1814 , Paris fell , and in April Napoleon surrendered .',
 'target': 'In early 1814 , Allied armies entered France , Paris surrendered , and Napoleon fell in April .'}

#Tokenization

In [10]:
model_name = 'T5-base'
token = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
max_length = max([len(token.encode(x)) for x in train_df['input_data']])

In [23]:
# Tokenization function
def token_fun(batch):
    # Tokenize input and target data
    inputs = token(batch['input_data'], max_length=max_length, padding='max_length', truncation=True)
    targets = token(batch['target'], max_length=max_length, padding='max_length', truncation=True)
    inputs['labels'] = targets['input_ids']  # Ensure correct key for labels
    return inputs

# Map tokenization function to datasets
token_train = train_data.map(token_fun, batched=True)
token_valid = valid_data.map(token_fun, batched=True)


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

#T5 fine tune

In [26]:
# Define training arguments for T5 model fine-tuning
training_args = TrainingArguments(
    output_dir='./t5_output',              # Directory where the fine-tuned model and checkpoints will be saved
    evaluation_strategy="steps",           # Evaluate model at regular intervals (steps)
    learning_rate=3e-5,                    # Learning rate for the optimizer
    per_device_train_batch_size=8,         # Batch size for training
    per_device_eval_batch_size=8,          # Batch size for evaluation
    num_train_epochs=10,                   # Number of epochs for training
    weight_decay=0.01,                     # Weight decay to reduce overfitting
    warmup_steps=500,                      # Number of warm-up steps for the learning rate scheduler
    save_steps=500,                        # Frequency (in steps) at which to save model checkpoints
    logging_dir='./logs',                  # Directory to store logs for monitoring training process
    logging_steps=100,                     # Frequency (in steps) to log training metrics
    eval_steps=500,                        # Frequency (in steps) for evaluation
    save_total_limit=3,                    # Limit on the number of saved model checkpoints
    metric_for_best_model='eval_loss',          # Metric to track and save the best model (BLEU score)
    report_to='wandb',                     # Optionally report training progress to WandB
    push_to_hub=False                      # Whether to push the model to the Hugging Face Model Hub (set to False if not needed)
)


trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=token_train,
    eval_dataset=token_valid

)
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.129200,0.274286
1000,0.160300,0.255745
1500,0.159300,0.251022
2000,0.157600,0.248384
2500,0.151100,0.248404
3000,0.145800,0.250905
3500,0.139500,0.251242


TrainOutput(global_step=3750, training_loss=0.15493496424357098, metrics={'train_runtime': 1595.3638, 'train_samples_per_second': 18.804, 'train_steps_per_second': 2.351, 'total_flos': 2676084480000000.0, 'train_loss': 0.15493496424357098, 'epoch': 10.0})

#result

In [47]:
import torch


# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the pre_text function
def pre_text(sentence):
    return 'paraphrase: ' + sentence

# Define the generate_paraphrase function
def generate_paraphrase(
    input_text, model, tokenizer, max_length=128, num_beams=4, num_return_sequences=4,
    top_k=100, top_p=0.9, temperature=1.0
):
    # Prepend the paraphrase prefix
    input_text = pre_text(input_text)

    # Tokenize input
    inputs = token(
        input_text, max_length=max_length, truncation=True,
        padding='max_length', return_tensors='pt'
    ).to(device)

    # Generate paraphrases
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length + 20,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
        do_sample=True,
        early_stopping=True
    )

    # Decode generated outputs
    paraphrased_texts = [token.decode(output, skip_special_tokens=True) for output in outputs]
    return paraphrased_texts




#Example

In [48]:
# Example usage
input_sentence = "The quick brown fox jumps over the lazy dog."
paraphrases = generate_paraphrase(input_sentence, model, token)
for idx, p in enumerate(paraphrases):
    print(f"Paraphrase {idx + 1}: {p}")


Paraphrase 1: The quick fox jumps over the lazy dog.
Paraphrase 2: The lazy dog jumps over the quick brown fox.
Paraphrase 3: The lazy fox jumps over the quick brown dog.
Paraphrase 4: 


In [51]:
# Example usage
input_sentence = "He enjoys playing soccer with his friends."
paraphrases = generate_paraphrase(input_sentence, model, token)
for idx, p in enumerate(paraphrases):
    print(f"Paraphrase {idx + 1}: {p}")

Paraphrase 1: He enjoys playing soccer with his friends.
Paraphrase 2: He enjoys playing with his friends soccer.
Paraphrase 3: He loves playing soccer with his friends.
Paraphrase 4: 
